A model on $n$ vertices with order $d$.
$K$ vertices forming a clique, meaning that any size-d tuple within the clique is connected by a hyperedge.
All other size-$d$ tuples form a hyperedge with probability $q = 1/2$ .

In [430]:
import math
import time as tm
from itertools import permutations

import numpy as np

In [431]:
# increment the degree of each vertex on graph g by 1
def add_all(i, j, k, g):
    g[i, j, k] = 1
    g[i, k, j] = 1
    g[j, i, k] = 1
    g[j, k, i] = 1
    g[k, i, j] = 1
    g[k, j, i] = 1
    return g

# decrement the degree of each vertex on graph g to 0
def remove_all(i, j, k, g):
    g[i, j, k] = 0
    g[i, k, j] = 0
    g[j, i, k] = 0
    g[j, k, i] = 0
    g[k, i, j] = 0
    g[k, j, i] = 0
    return g

In [432]:
### create a graph with planted clique

# g: graph (represented as a tensor)
# num_vertices: number of vertices on the graph (dimension, or rank of the tensor) 
# pr: probability of creating edges
# clq_size: clique size

def generate_graph(num_vertices, pr, clq_size):
    def create_edges():
        for i in range(num_vertices):
            for j in range(i + 1, num_vertices):
                for k in range(j + 1, num_vertices):
                    if np.random.uniform(0, 1, 1) < pr:
                        vec[[i, j, k]] += 1
                        add_all(i, j, k, g)

    def plant_clique():
        for i in range(clq_size):
            for j in range(i + 1, clq_size):
                for k in range(j + 1, clq_size):
                    vec[[clq_vertex[i], clq_vertex[j], clq_vertex[k]]] += 1
                    add_all(clq_vertex[i], clq_vertex[j], clq_vertex[k], g)

    g = np.zeros((num_vertices, num_vertices, num_vertices))
    vec = np.zeros(num_vertices, )
    create_edges()
    clq_vertex = np.random.choice(range(num_vertices), clq_size, replace=False)
    plant_clique()

    return g, vec, clq_vertex

In [433]:
# calculate the total number of edges on a graph with num_vertices vertices
def count_edge(g, num_vertices):
    num_edges = 0
    for i in range(num_vertices):
        for j in range(i + 1, num_vertices):
            for k in range(j + 1, num_vertices):
                if g[i, j, k] == 1:
                    num_edges += 1
    return num_edges

In [434]:
# check if the graph is a clique
def is_clique(g, vec, num_vertices):
    active_count = np.count_nonzero(vec)  # number of vertices that are associated with at least 1 edge
    edge_sum = count_edge(g, num_vertices)  # number of edges
    return edge_sum == math.comb(active_count, 3)

In [435]:
# remove edge
def remove_edges(g, num_vertices, vec, curr_idx):
    vec[curr_idx] = 0
    for j in range(0, num_vertices):
        for k in range(j+1, num_vertices):
            if g[curr_idx, j, k] == 1:
                vec[[j, k]] -= 1
                remove_all(curr_idx, j, k, g)
    return g, vec

### Generate a graph
N: number of vertices
P: probability of an edge being included
K: clique size

G: graph with planted clique
V: vector storing the number of edges associated with each vertex in the graph
clique_vertices: the set of clique vertices

In [436]:
N = 100  # number of vertices
P = 0.5  # probability of an edge being included
K = 10  # clique size

start_generate = tm.time()
res = generate_graph(N, P, K)
G, V, clique_vertices = res[0], res[1], res[-1]
G_0 = G.copy()

print(f"edge-occurrence vector: {V}\n")
print(f"set of clique vertices after generation phase: {clique_vertices}\n")

time_generate = np.round(tm.time() - start_generate, 3)
# print("Time taken to generate graph: ", "seconds")

edge-occurrence vector: [2447. 2455. 2462. 2424. 2418. 2458. 2393. 2431. 2468. 2459. 2439. 2397.
 2446. 2450. 2455. 2414. 2392. 2500. 2488. 2464. 2461. 2403. 2475. 2422.
 2457. 2454. 2467. 2428. 2420. 2440. 2409. 2399. 2336. 2438. 2419. 2483.
 2440. 2348. 2413. 2443. 2395. 2502. 2375. 2482. 2385. 2441. 2435. 2422.
 2456. 2387. 2495. 2446. 2459. 2504. 2428. 2407. 2418. 2372. 2413. 2419.
 2400. 2448. 2415. 2400. 2394. 2470. 2453. 2440. 2408. 2406. 2473. 2379.
 2422. 2434. 2483. 2448. 2388. 2460. 2436. 2443. 2470. 2429. 2389. 2469.
 2470. 2477. 2425. 2427. 2476. 2484. 2442. 2443. 2412. 2427. 2402. 2428.
 2442. 2344. 2392. 2396.]

set of clique vertices after generation phase: [33 17 52 36  8 70 74 19 67  1]



In [437]:
# V

### Removal Phase

In [438]:
start_removal = tm.time()
itr = 0
removed = []  # keep track of the vertices removed from the original graph to form a clique

while not is_clique(G, V, N):
    itr += 1
    curr = -1
    idx_sorted = np.argsort(V)

    for idx in range(N):
        # print(V)
        if V[idx_sorted[idx]] != 0:
            # print(idx_sorted[idx])
            curr = idx_sorted[idx]
            removed.append(curr)
            # V[curr] = 0
            break

    A = remove_edges(G, N, V, curr)
    
    G, V = A[0], A[1]
    print(f"\nupdated: {V}\n")
    # N = np.count_nonzero(V)

print(f"is a clique at iteration #{itr}!!!")
time_remove = np.round(tm.time() - start_removal, 3)


updated: [2399. 2403. 2416. 2376. 2372. 2409. 2351. 2387. 2426. 2413. 2406. 2351.
 2399. 2402. 2403. 2373. 2350. 2451. 2442. 2420. 2415. 2352. 2431. 2373.
 2412. 2397. 2420. 2375. 2365. 2395. 2360. 2354.    0. 2389. 2374. 2437.
 2399. 2306. 2360. 2399. 2351. 2451. 2332. 2435. 2329. 2390. 2386. 2381.
 2409. 2339. 2441. 2400. 2418. 2458. 2369. 2363. 2377. 2324. 2359. 2375.
 2353. 2399. 2364. 2362. 2347. 2420. 2402. 2397. 2359. 2356. 2414. 2324.
 2385. 2386. 2432. 2402. 2352. 2410. 2382. 2400. 2417. 2372. 2343. 2427.
 2412. 2428. 2378. 2386. 2431. 2434. 2396. 2396. 2371. 2381. 2358. 2378.
 2396. 2303. 2347. 2343.]


updated: [2353. 2363. 2371. 2335. 2325. 2362. 2304. 2344. 2384. 2364. 2357. 2306.
 2347. 2351. 2360. 2323. 2305. 2405. 2385. 2372. 2368. 2314. 2378. 2325.
 2361. 2356. 2370. 2334. 2313. 2347. 2311. 2307.    0. 2345. 2330. 2381.
 2353. 2255. 2309. 2346. 2309. 2410. 2292. 2389. 2288. 2337. 2339. 2339.
 2361. 2294. 2394. 2354. 2363. 2402. 2327. 2314. 2325. 2275. 2306. 2331.
 230

In [439]:
print(f"number of iterations in the removal phase: {itr}")

number of iterations in the removal phase: 91


In [440]:
included = np.setdiff1d(np.arange(N), removed)
# included

### Inclusion Phase

In [441]:
included

array([ 1,  8, 17, 19, 33, 36, 52, 70, 74])

In [442]:
# removed

In [443]:
def inclusion_phase(target, in_set, g):
    def connected():
        for j in range(len(in_set)):
            for k in range(j + 1, len(in_set)):
                if g[target_idx, in_set[j], in_set[k]] != 1:
                    # print(f" vertex {idx} is not connected to all clique vertices")
                    return False
        return True

    for target_idx in target:
        if connected():
            print(f"add {target_idx} to clique!")
            in_set = np.append(in_set, target_idx)
            print(in_set)

    return in_set

In [444]:
start_include = tm.time()
res = inclusion_phase(removed, included, G_0)
time_include = np.round(tm.time() - start_include, 3)

add 67 to clique!
[ 1  8 17 19 33 36 52 70 74 67]


In [445]:
# print(f"Set of clique vertices after generation phase: {np.sort(clique_vertices)}\n")
print(f"Set of clique vertices after removal phase: {np.sort(included)}\n")
print(f"Set of clique vertices after inclusion phase: {np.sort(res)}")

Set of clique vertices after removal phase: [ 1  8 17 19 33 36 52 70 74]

Set of clique vertices after inclusion phase: [ 1  8 17 19 33 36 52 67 70 74]
